# Thesis 2020-2021: Word2Vec (word embeddings)

In this notebook, we will create a Word2Vec model.

In [4]:
import pandas as pd
import numpy as np
import math

import matplotlib
import matplotlib.pyplot as plt

from gensim.models import Word2Vec

In [48]:
import re
from pattern.text.en import singularize
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

tokenizer = TweetTokenizer()
stop_words = set(stopwords.words('english'))

# Create a function to clean the tweets
def cleanTxt(text):
    text = text.lower() # Convert everything to lower case
    text = re.sub(r'@[a-zA-Z0-9]+', '', text) # Remove @mentions
    text = re.sub(r'rt[\s]+', '', text) # Remove RT (retweet symbol)
    text = re.sub(r'&amp;', 'and', text) # Replace '&amp;' by 'and'
    text = re.sub(r'https?:\/\/\S+', '', text) # Remove hyper link  
    #text = re.sub(r'\d+', '0', text) # Replace all numbers by a zero
    text = " ".join([singularize(word) for word in tokenizer.tokenize(text) if word not in stop_words]) # Remove stopwords
    #text = " ".join([singularize(word) for word in text])
    text = re.sub(r'[^\w\s#]', ' ', text) # Remove all non-alphanumeric symbols (excluding whitespace and # characters)
    text = re.sub(r'\s+', ' ', text) # Replace multiple whitespaces by a single whitespace
    text = text.strip() # Remove whitespaces at the beginning and at the end
    
    return text

In [49]:
import csv
    
df_train = pd.read_csv('data/hateval2019_en_train.csv')
df_dev = pd.read_csv('data/hateval2019_en_dev.csv')

df_train_dev = df_train.append(df_dev, ignore_index=True)
df_train_dev = df_train_dev.drop(['TR', 'AG'], axis=1)

# Clean the data

df_train_dev['text_cleaned'] = df_train_dev['text'].apply(cleanTxt)
df_test['text_cleaned'] = df_test['text'].apply(cleanTxt)
df_train_dev

,id,text,HS,text_cleaned
0,201,"Hurray, saving us $$$ in so many ways @potus @...",1,hurray saving u many way #lockthemup #buildthe...
1,202,Why would young fighting age men be the vast m...,1,would young fighting age man vast majority one...
2,203,@KamalaHarris Illegals Dump their Kids at the ...,1,illegal dump kid border like road kill refuse ...
3,204,NY Times: 'Nearly All White' States Pose 'an A...,0,ny time s nearly white s state pose s array pr...
4,205,Orban in Brussels: European leaders are ignori...,0,orban brussel european leader ignoring person ...
...,...,...,...,...
9995,19196,@SamEnvers you unfollowed me? Fuck you pussy,0,unfollowed fuck pussy
9996,19197,@DanReynolds STFU BITCH! AND YOU GO MAKE SOME ...,1,stfu bitch go make satanic music u illuminatus...
9997,19198,"@2beornotbeing Honey, as a fellow white chick,...",0,honey fellow white chick let tell need shut fu...
9998,19199,I hate bitches who talk about niggaz with kids...,1,hate bitch talk niggaz kid everybody cant find...


In [6]:
df_test = pd.read_csv('data/hateval2019_en_test.csv')
df_test = df_test.drop(['TR', 'AG'], axis=1)
df_test

,id,text,HS
0,34243,"@local1025 @njdotcom @GovMurphy Oh, I could ha...",0
1,30593,Several of the wild fires in #california and #...,0
2,31427,@JudicialWatch My question is how do you reset...,0
3,31694,"#Europe, you've got a problem! We must hurry...",1
4,31865,This is outrageous! #StopIllegalImmigration #...,1
...,...,...,...
2995,31368,you can never take a L off a real bitch😩 im ho...,1
2996,30104,@Brian_202 likes to call me a cunt & a bitch b...,1
2997,31912,@kusha1a @Camio_the_wise @shoe0nhead 1. Never ...,1
2998,31000,If i see and know you a hoe why would i hit yo...,1


In [127]:
X_train = df_train_dev.text_cleaned
X_test = df_test.text_cleaned
y_train = df_train_dev.HS
y_test = df_test.HS

In [112]:
train_sentences = [tokenizer.tokenize(s) for s in X_train]
test_sentences = [tokenizer.tokenize(s) for s in X_test]
train_sentences

[['hurray',
  'saving',
  'u',
  'many',
  'way',
  '#lockthemup',
  '#buildthewall',
  '#enddaca',
  '#boycottnfl',
  '#boycottnike'],
 ['would',
  'young',
  'fighting',
  'age',
  'man',
  'vast',
  'majority',
  'one',
  'escaping',
  'war',
  'cannot',
  'fight',
  'like',
  'woman',
  'child',
  'elderly',
  'majority',
  'refugee',
  'actually',
  'refugee',
  'economic',
  'migrant',
  'trying',
  'get',
  'europe'],
 ['illegal',
  'dump',
  'kid',
  'border',
  'like',
  'road',
  'kill',
  'refuse',
  'unite',
  'hope',
  'get',
  'amnesty',
  'free',
  'education',
  'welfare',
  'illegal',
  '#familesbelongtogether',
  'country',
  'taxpayer',
  'dime',
  'scam',
  '#nodaca',
  '#noamnesty',
  '#sendthe'],
 ['ny',
  'time',
  's',
  'nearly',
  'white',
  's',
  'state',
  'pose',
  's',
  'array',
  'problem',
  's',
  'immigrant'],
 ['orban',
  'brussel',
  'european',
  'leader',
  'ignoring',
  'person',
  'want',
  'migrant'],
 ['legal', 'illegal', '#buildthatwall'],
 

In [53]:
model = Word2Vec(sentences=train_sentences, vector_size=100, window=5, min_count=1, workers=4)
model.wv.index_to_key
#vector = model.wv['saving']  # get numpy vector of a word
#sims = model.wv.most_similar('saving', topn=10)  # get other similar words


['woman',
 's',
 'bitch',
 'refugee',
 'immigrant',
 'u',
 'like',
 'migrant',
 'illegal',
 'man',
 'person',
 'get',
 'immigration',
 'rape',
 'go',
 'want',
 'cunt',
 'country',
 'one',
 'fuck',
 'whore',
 'time',
 'hysterical',
 'say',
 '#buildthatwall',
 'know',
 'right',
 'girl',
 'need',
 'fucking',
 'i',
 'child',
 'home',
 'as',
 'make',
 'stop',
 'back',
 'year',
 'hoe',
 'vium',
 'trump',
 'would',
 'take',
 'skank',
 'pussy',
 'new',
 'border',
 'think',
 'stupid',
 'life',
 'dick',
 '#maga',
 'm',
 'family',
 'never',
 '#buildthewall',
 'come',
 'shit',
 'american',
 'good',
 'muslim',
 'see',
 'white',
 'world',
 'let',
 'help',
 'free',
 'law',
 'many',
 't',
 '2',
 'europe',
 'even',
 'hole',
 'slut',
 'eu',
 'must',
 'dont',
 'look',
 'call',
 'way',
 'alien',
 'love',
 '1',
 'first',
 'shut',
 'got',
 'kid',
 'day',
 'tell',
 '#trump',
 '#immigration',
 'really',
 'give',
 'work',
 'that',
 'old',
 'still',
 'migration',
 'thing',
 'america',
 '#nodaca',
 'stfu',
 'sai

In [76]:
vector = model.wv['woman']  # get numpy vector of a word
len(vector)

100

In [72]:
model.wv.most_similar('man', topn=10)  # get other similar words

[('woman', 0.9995855689048767),
 ('hysterical', 0.9995812177658081),
 ('say', 0.9995176792144775),
 ('one', 0.9994102716445923),
 ('thing', 0.9993219375610352),
 ('rape', 0.9993157386779785),
 ('think', 0.9993077516555786),
 ('white', 0.9992996454238892),
 ('said', 0.9992806315422058),
 ('assault', 0.9992477893829346)]

In [125]:
# Function that creates Word2Vec vectors for each sentences (by taking the average of the word2vecs of each word within sentence)

def create_word2vec_vectors(sentences):
    list_vectors = []
    for s in sentences:
        temp = np.zeros(100)
        if (len(s) == 0):
            list_vectors.append(list(temp))
            continue
        for word in s:
            if (word not in model.wv.index_to_key): continue # This condition is needed because there could be some unseen words within the test data
            temp = temp + model.wv[word]
        list_vectors.append(list(temp/len(s)))
    
    return list_vectors

In [126]:
train_vectors = create_word2vec_vectors(train_sentences)
print(np.shape(train_vectors))
test_vectors = create_word2vec_vectors(test_sentences)
print(np.shape(test_vectors))

(10000, 100)
(3000, 100)


In [106]:
# EXTRA: Dubble check if you calculated the vectors correctly

even_testen = model.wv[train_sentences[0][0]]
for w in train_sentences[0][1:]:
    even_testen = even_testen + model.wv[w]

(even_testen/len(train_sentences[0]))

array([-3.30899686e-01,  4.45880026e-01,  2.44529933e-01, -2.62317538e-01,
       -4.91707548e-02, -6.60078883e-01,  2.52522647e-01,  7.55166054e-01,
       -2.49226615e-02, -4.36568022e-01, -7.01646656e-02, -6.20993733e-01,
       -5.40604368e-02, -1.81502104e-01,  2.04280645e-01, -5.28267562e-01,
        1.32858917e-01, -6.20416760e-01,  1.33758217e-01, -6.45467162e-01,
        7.38661736e-02,  5.13638258e-02,  5.09726167e-01, -2.48283193e-01,
       -7.04475045e-01,  1.15412712e-01, -5.08204103e-01, -2.44777530e-01,
       -2.56729513e-01,  2.41511315e-01,  4.97765243e-01, -4.58144173e-02,
        1.16910435e-01, -1.21255159e-01,  6.57712296e-02,  5.36842942e-01,
       -8.95184278e-03, -3.08947057e-01, -2.84584582e-01, -6.42600954e-01,
        2.12115452e-01, -2.03791618e-01, -4.44506705e-02,  6.66731894e-02,
        5.39922476e-01, -8.10264498e-02,  1.34869786e-02,  1.00230351e-01,
        1.93458289e-01,  3.18110377e-01,  3.01467478e-01, -6.22003973e-02,
        6.80165440e-02, -

## Now we will evaluate the Word2Vec model using a LR model as classifier
- Evaluate without normalizing/scaling data
- Evaluate after normalizing/scaling data

In [128]:
# Evaluate the Word2Vec model using Logistic Regression as the classifier (without normalizing input data)

import import_ipynb
import evaluate # here we import the local evaluate.ipynb jupyter notebook
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression().fit(train_vectors, y_train)
y_predict = logreg.predict(test_vectors)

# Create new test dataframe
df_test_word2vec = df_test.copy()
df_test_word2vec['HS'] = y_predict

# Create prediction file for the word2vec
df_test_word2vec[['id', 'HS']].to_csv('predictions/word2vec.tsv', sep='\t', index=False, header=False)
df_test_word2vec[['id', 'HS']].to_csv('input/res/en_a.tsv', sep='\t', index=False, header=False)

# Evaluate the result of the word2vec
evaluate.write_eval("scores_word2vec")

importing Jupyter notebook from evaluate.ipynb
taskA_fscore: 0.5068050317255326
taskA_precision: 0.5324273598832443
taskA_recall: 0.5308292282430214
taskA_accuracy: 0.5083333333333333


In [130]:
# Normalize the data via StandardScaler

from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train_vectors)
train_vectors_scaled = scaler.transform(train_vectors)
test_vectors_scaled = scaler.transform(test_vectors)

#clf_scaled = LogisticRegression().fit(X_train_scaled, y_train)
#y_scaled_predict = clf_scaled.predict(X_test_scaled)

In [131]:
# Evaluate the bigram model using Logistic Regression as the classifier (+ normalizing input data)

import import_ipynb
import evaluate # here we import the local evaluate.ipynb jupyter notebook
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression().fit(train_vectors_scaled, y_train)
y_predict_scaled = logreg.predict(test_vectors_scaled)

# Create new test dataframe
df_test_word2vec_scaled = df_test.copy()
df_test_word2vec_scaled['HS'] = y_predict_scaled

# Create prediction file for the word2vec_scaled
df_test_word2vec_scaled[['id', 'HS']].to_csv('predictions/word2vec_scaled.tsv', sep='\t', index=False, header=False)
df_test_word2vec_scaled[['id', 'HS']].to_csv('input/res/en_a.tsv', sep='\t', index=False, header=False)

# Evaluate the result of the word2vec_scaled
evaluate.write_eval("scores_word2vec_scaled")

taskA_fscore: 0.48115551147236035
taskA_precision: 0.5339535906656155
taskA_recall: 0.5279556650246305
taskA_accuracy: 0.49166666666666664


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [132]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score , average='macro')
params = {'C': [0.001, 0.01, 0.1, 1, 10], 'solver': ('newton-cg', 'lbfgs', 'liblinear'), 'penalty': ('l1', 'l2', 'elasticnet')}
grid = GridSearchCV(estimator=LogisticRegression(max_iter=500), param_grid=params, cv=5, scoring=f1, verbose=5, n_jobs=5)
grid.fit(train_vectors, y_train)
print("Best cross-validation score: ", grid.best_score_)
print("Best parameters: ", grid.best_params_)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:   34.2s
[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed:  4.1min
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed: 10.5min
[Parallel(n_jobs=5)]: Done 225 out of 225 | elapsed: 21.3min finished


Best cross-validation score:  0.6883430897988031
Best parameters:  {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}


In [134]:
# Evaluate the Word2Vec model using optimized Logistic Regression as the classifier

import import_ipynb
import evaluate # here we import the local evaluate.ipynb jupyter notebook
from sklearn.linear_model import LogisticRegression

logreg_optimized = LogisticRegression(C=10, penalty='l1', solver='liblinear').fit(train_vectors, y_train)
y_predict_optimized = logreg_optimized.predict(test_vectors)

# Create new test dataframe
df_test_word2vec_optimized = df_test.copy()
df_test_word2vec_optimized['HS'] = y_predict_optimized

# Create prediction file for the word2vec_optimized
df_test_word2vec_optimized[['id', 'HS']].to_csv('predictions/word2vec_optimized.tsv', sep='\t', index=False, header=False)
df_test_word2vec_optimized[['id', 'HS']].to_csv('input/res/en_a.tsv', sep='\t', index=False, header=False)

# Evaluate the result of the word2vec_optimized
evaluate.write_eval("scores_word2vec")

taskA_fscore: 0.45887837800087117
taskA_precision: 0.526378220548468
taskA_recall: 0.5195812807881773
taskA_accuracy: 0.477


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
